In [11]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
import xgboost as xgb
from sklearn import metrics
import mlflow
import mlflow.sklearn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,r2_score

#### Définition de nos variables cibles 

In [12]:
data = pd.read_csv("../data/flight_data.csv")

In [13]:
Y = data['Price']
X = data.drop(['Price'], axis=1)

#### Divisez les données en ensembles d’entraînement et de test

In [14]:
X_train_or, X_test, y_train_or, y_test = train_test_split(X,Y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train_or, y_train_or, test_size=0.25)

In [15]:
mlflow.set_tracking_uri("sqlite:///mlflowtest.db")
mlflow.set_experiment("Flight price prediction tracking")

2023/12/15 13:40:04 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/12/15 13:40:04 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

<Experiment: artifact_location='file:///c:/Users/Dell/ENSIAS/S3/flight-price-prediction/notebooks/mlruns/1', creation_time=1702644005388, experiment_id='1', last_update_time=1702644005388, lifecycle_stage='active', name='Flight price prediction tracking', tags={}>

In [16]:
mlflow.end_run()

In [17]:
mlflow.start_run()
mlflow.log_param("test_size", 0.2)
mlflow.log_param("random_state", 42)

42

# Linear Regression

In [18]:
mlflow.end_run()

In [19]:
with mlflow.start_run(run_name='Linear regression model'):
    reg_model=LinearRegression().fit(X_train,y_train)
    y_pred_val =reg_model.predict(X_val)

    mse_val =mean_squared_error(y_val,y_pred_val)
    mae_val=metrics.mean_absolute_error(y_val,y_pred_val)
    rmse_val=np.sqrt(metrics.mean_absolute_error(y_val,y_pred_val))
    r2_val =r2_score(y_test,y_pred_val)
    
    val_metrics = {'r2_score': r2_val, 'mse':mse_val,'rmse':rmse_val,'mae':mae_val}
    mlflow.log_metrics(val_metrics)
    mlflow.sklearn.log_model(reg_model,"Linear regression model")
mlflow.end_run()
    

c:\Users\Dell\anaconda3\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\Dell\anaconda3\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


# Random Forest

In [20]:
with mlflow.start_run(run_name='Random Forest Regressor Model'):
    rf = RandomForestRegressor()
    rf.fit(X_train, y_train)
    y_pred_val_rf = rf.predict(X_val)

    mse_val_rf = mean_squared_error(y_val, y_pred_val_rf)
    mae_val_rf = metrics.mean_absolute_error(y_val, y_pred_val_rf)
    rmse_val_rf = np.sqrt(mean_squared_error(y_val, y_pred_val_rf))
    r2_val_rf = r2_score(y_val, y_pred_val_rf)

    val_metrics_rf = {'r2_score': r2_val_rf, 'mse': mse_val_rf, 'rmse': rmse_val_rf, 'mae': mae_val_rf}
    mlflow.log_metrics(val_metrics_rf)
    mlflow.sklearn.log_model(rf, "Random Forest Regressor Model")
mlflow.end_run()

c:\Users\Dell\anaconda3\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\Dell\anaconda3\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


# XG BOOST

In [21]:
with mlflow.start_run(run_name='XGBoost Regressor Model'):
    # Define hyperparameters
    xg_params = {
        'objective': 'reg:squarederror',
        'colsample_bytree': 0.3,
        'learning_rate': 0.1,
        'max_depth': 5,
        'alpha': 10,
        'n_estimators': 10
    }
    xg_reg = xgb.XGBRegressor(**xg_params) 

    xg_reg.fit(X_train, y_train)
    y_pred_val_xgb = xg_reg.predict(X_val)

    mse_val_xgb = mean_squared_error(y_val, y_pred_val_xgb)
    mae_val_xgb = metrics.mean_absolute_error(y_val, y_pred_val_xgb)
    rmse_val_xgb = np.sqrt(mean_squared_error(y_val, y_pred_val_xgb))
    r2_val_xgb = r2_score(y_val, y_pred_val_xgb)

    val_metrics_xgb = {'r2_score': r2_val_xgb, 'mse': mse_val_xgb, 'rmse': rmse_val_xgb, 'mae': mae_val_xgb}
    mlflow.log_metrics(val_metrics_xgb)
    mlflow.log_params(xg_params)
  
    mlflow.xgboost.log_model(xg_reg, "XGBoost Regressor Model")
    
mlflow.end_run()

c:\Users\Dell\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [13:43:13] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0750514818a16474a-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)
c:\Users\Dell\anaconda3\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\Dell\anaconda3\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: 

# Decision Tree

In [22]:
with mlflow.start_run(run_name='Decision Tree Model'):
    dt_model = DecisionTreeRegressor()

    dt_model.fit(X_val, y_val)
    y_pred_val = dt_model.predict(X_val)


    mse_val =mean_squared_error(y_val,y_pred_val)
    mae_val=metrics.mean_absolute_error(y_val,y_pred_val)
    rmse_val=np.sqrt(metrics.mean_absolute_error(y_val,y_pred_val))
    r2_val =r2_score(y_test,y_pred_val)


    val_metrics = {'r2_score': r2_val, 'mse':mse_val,'rmse':rmse_val,'mae':mae_val}
    mlflow.log_metrics(val_metrics)
    mlflow.sklearn.log_model(dt_model,"Decision TreeRegressor model")
mlflow.end_run()

c:\Users\Dell\anaconda3\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\Dell\anaconda3\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
